In [53]:
!pip3 install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 59.2 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/EhsanAghazadeh/bert-based-uncased-sst2-e1

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/EhsanAghazadeh/bert-based-uncased-sst2-e1)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="EhsanAghazadeh/bert-based-uncased-sst2-e1")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/876 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset,DataLoader
from transformers import BertTokenizer,BertModel
# import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from pylab import rcParams
# from torchmetrics.functional import accuracy
from transformers import get_linear_schedule_with_warmup,get_constant_schedule,get_cosine_schedule_with_warmup
from matplotlib import rc

In [54]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset,DataLoader
from transformers import BertTokenizer,BertModel
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from pylab import rcParams
# from torchmetrics.functional import accuracy
from transformers import get_linear_schedule_with_warmup,get_constant_schedule,get_cosine_schedule_with_warmup
from matplotlib import rc


In [23]:
df=pd.read_csv("sample_submission.csv",on_bad_lines="skip")
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [24]:
train_df,val_df=train_test_split(df,test_size=0.2,random_state=42)
LABEL_COLUMNS=['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [25]:
train_df[LABEL_COLUMNS].sum()
train_df=train_df.sample(100_000)
train_df[LABEL_COLUMNS].sum()

,0
toxic,9646
severe_toxic,1000
obscene,5257
threat,319
insult,4887
identity_hate,858


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("EhsanAghazadeh/bert-based-uncased-sst2-e1")
model = AutoModelForSequenceClassification.from_pretrained("EhsanAghazadeh/bert-based-uncased-sst2-e1")
encoding=tokenizer.encode_plus(sample_comment,add_special_tokens=True,return_tensors="pt")
encoding

In [27]:
sample_row = df.iloc[16]

# Access the 'comment_text' column
sample_comment = sample_row['comment_text']

In [34]:

encoding['input_ids'].shape
encoding['input_ids'].squeeze()[:20]

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'].squeeze().tolist())
tokens[:20]

['[CLS]',
 'bye',
 '!',
 'don',
 "'",
 't',
 'look',
 ',',
 'come',
 'or',
 'think',
 'of',
 'com',
 '##ming',
 'back',
 '!',
 'toss',
 '##er',
 '.',
 '[SEP]']

In [36]:
class ToxicCommentDataset(Dataset):
  def __init__(self,data:pd.DataFrame,tokenizer:BertTokenizer,max_len:int=128):
    self.tokenizer=tokenizer
    self.data=data
    self.max_len=max_len
  def __len__(self):
    return len(self.data)
  def __getitem__(self,index:int):
    data_row=self.data.iloc[index]
    comment=data_row.comment_text
    labels=data_row[LABEL_COLUMNS]
    encoding=self.tokenizer.encode_plus(comment,add_special_tokens=True)
    return dict(comment=comment,input_ids=encoding['input_ids'],attention_mask=encoding['attention_mask'],labels=torch.FloatTensor(labels))


In [44]:
train_dataset=ToxicCommentDataset(train_df,tokenizer)
sample_item=train_dataset[0]
sample_item.keys()
sample_item['comment']
sample_item['labels']
# sample_item['input_ids'].shape[0]

/tmp/ipython-input-2949199063.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dict(comment=comment,input_ids=encoding['input_ids'],attention_mask=encoding['attention_mask'],labels=torch.FloatTensor(labels))


tensor([0., 0., 0., 0., 0., 0.])

In [49]:
MODEL_NAME = "EhsanAghazadeh/bert-based-uncased-sst2-e1"
bert_model = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
sample_item=train_dataset[0]
sample_item.keys()
prediction=bert_model(input_ids=torch.tensor(sample_item['input_ids']).unsqueeze(dim=0),attention_mask=torch.tensor(sample_item['attention_mask']).unsqueeze(dim=0))
prediction.last_hidden_state.shape,prediction.pooler_output.shape

/tmp/ipython-input-2949199063.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dict(comment=comment,input_ids=encoding['input_ids'],attention_mask=encoding['attention_mask'],labels=torch.FloatTensor(labels))


ValueError: You need to specify either `text` or `text_target`.

In [51]:

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
MODEL_NAME = "EhsanAghazadeh/bert-based-uncased-sst2-e1"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load base BERT model (to get last_hidden_state, pooler_output)
bert_model = AutoModel.from_pretrained(MODEL_NAME, return_dict=True)

# (Optional) Load classification model if you want logits
clf_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Example: take one sample from your dataset
sample_item = train_dataset[0]
print(sample_item.keys())  # e.g., {'input_ids', 'attention_mask', 'comment_text'}

# Convert to tensors
input_ids = torch.tensor(sample_item['input_ids']).unsqueeze(0)
attention_mask = torch.tensor(sample_item['attention_mask']).unsqueeze(0)

# Run through base BERT
with torch.no_grad():
    prediction = bert_model(input_ids=input_ids, attention_mask=attention_mask)

print("last_hidden_state shape:", prediction.last_hidden_state.shape)
print("pooler_output shape:", prediction.pooler_output.shape)

# --- If you want classification logits instead ---
with torch.no_grad():
    clf_outputs = clf_model(input_ids=input_ids, attention_mask=attention_mask)
print("logits shape:", clf_outputs.logits.shape)

/tmp/ipython-input-2949199063.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dict(comment=comment,input_ids=encoding['input_ids'],attention_mask=encoding['attention_mask'],labels=torch.FloatTensor(labels))


dict_keys(['comment', 'input_ids', 'attention_mask', 'labels'])
last_hidden_state shape: torch.Size([1, 95, 768])
pooler_output shape: torch.Size([1, 768])
logits shape: torch.Size([1, 2])


In [55]:
class ToxicCommentDataModule(pl.LightningDataModule):
  def __init__(self,train_df:pd.DataFrame,test_df:pd.DataFrame,tokenizer,batch_size=8,max_token_len=128):
    super().__init__()
    self.train_df=train_df
    self.test_df=test_df
    self.tokenizer=tokenizer
    self.batch_size=batch_size
    self.max_token_len=max_token_len
  def setup(self,stage=None):
    self.train_dataset=ToxicCommentDataset(self.train_df,self.tokenizer)
    self.test_dataset=ToxicCommentDataset(self.test_df,self.tokenizer)
  def train_dataloader(self):
    return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True,num_workers=4)
  def val_dataloader(self):
    return DataLoader(self.test_dataset,batch_size=self.batch_size,shuffle=True,num_workers=4)
  def test_dataloader(self):
    return DataLoader(self.test_dataset,batch_size=self.batch_size,shuffle=True,num_workers=4)

In [57]:
data_module=ToxicCommentDataModule(train_df,val_df,tokenizer)
data_module.setup()